# Phone Conditions by Ad

In [1]:
# going to try sorting out the phone conditions out by individual ad
# libraries
import cv2
import numpy as np
import pandas as pd
import os
import csv
import av
import subprocess
import matplotlib.pyplot as plt
import glob
from datetime import datetime
from PIL import Image, ImageDraw
from scipy.interpolate import interp1d
import re

In [2]:
# paths and files for subj looping
t7sheild = os.path.join('/Volumes/T7 Shield/fox')
subj_folders = glob.glob(os.path.join('/Volumes/T7 Shield/fox/et','sub-[0-9][0-9][0-9]_phone-*'))
subj_folders = [folder for folder in subj_folders if 'control' not in folder]
subj_folders.sort()
print(len(subj_folders))
subj_folders

39


['/Volumes/T7 Shield/fox/et/sub-095_phone-a-87a1bd3d',
 '/Volumes/T7 Shield/fox/et/sub-096_phone-b-3611b179',
 '/Volumes/T7 Shield/fox/et/sub-097_phone-b-ebeeac21',
 '/Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7',
 '/Volumes/T7 Shield/fox/et/sub-099_phone-b-55ff7c53',
 '/Volumes/T7 Shield/fox/et/sub-100_phone-b-66bd4dcc',
 '/Volumes/T7 Shield/fox/et/sub-104_phone-b-4cafddda',
 '/Volumes/T7 Shield/fox/et/sub-105_phone-a-448ad877',
 '/Volumes/T7 Shield/fox/et/sub-106_phone-b-1d313c9f',
 '/Volumes/T7 Shield/fox/et/sub-107_phone-a-6661b0ea',
 '/Volumes/T7 Shield/fox/et/sub-109_phone-a-463e65ac',
 '/Volumes/T7 Shield/fox/et/sub-110_phone-b-a96e2678',
 '/Volumes/T7 Shield/fox/et/sub-111_phone-a-f197f0ab',
 '/Volumes/T7 Shield/fox/et/sub-115_phone-a-f60825c4',
 '/Volumes/T7 Shield/fox/et/sub-116_phone-a-d9c05810',
 '/Volumes/T7 Shield/fox/et/sub-117_phone-b-4d98cbb8',
 '/Volumes/T7 Shield/fox/et/sub-119_phone-b-b2c5cdc2',
 '/Volumes/T7 Shield/fox/et/sub-158_phone-a-0ef92d52',
 '/Volumes

In [4]:
# paths and files for testing
# locally
# subj_folder = os.path.join('data', 'sub-053_tv-lego-63478be7')
# video = os.path.join(subj_folder, 'a5ba9c89_0.0-813.692.mp4')
# events = pd.read_csv(os.path.join(subj_folder,'events.csv'))
# gaze = pd.read_csv(os.path.join(subj_folder,'gaze.csv'))
# world_timestamps = pd.read_csv(os.path.join(subj_folder,'world_timestamps.csv'))
# task_events = pd.read_csv('/Users/ebeard/Dropbox (Penn)/i3/foxmedia/data/tv_event/sub-053_lego/sub-053_task-events.csv')

# on t7shield
subj_folder = subj_folders[2]
big_video = glob.glob(os.path.join(subj_folder, '*.mp4'))[0]

# Log Errors

In [3]:
def log_error(error_message, subj=None):
    today_date = datetime.now().strftime("%Y-%m-%d")
    log_filename = f"error_log-phone-{today_date}.txt"
    with open(log_filename, "a") as log_file:
        if subj:
            log_file.write(f"Subject: {subj}\n")
        log_file.write(f"Error: {error_message}\n\n")

# Subset Individal Ads
We don't need to align the pupil labs events because we've decided to hand-code instead.

In [6]:
# testing
subj = subj_folder.split('/')[5].split('_')[0]
cond = subj_folder.split('/')[5].split('-')[2]

events = pd.read_csv(os.path.join(subj_folder,'events.csv'))
world_timestamps = pd.read_csv(os.path.join(subj_folder,'world_timestamps.csv'))
gaze = pd.read_csv(os.path.join(subj_folder,'gaze.csv'))

## Subset Ad Events

In [4]:
def create_event_onsets_and_durations(events_df, world_timestamps_df):

    # Ensure events_df and world_timestamps_df are DataFrames
    if not isinstance(events_df, pd.DataFrame):
        events_df = pd.DataFrame(events_df)

    if not isinstance(world_timestamps_df, pd.DataFrame):
        world_timestamps_df = pd.DataFrame(world_timestamps_df)
        
    # Filter events for start and end using regex to match a digit followed by .start or .end
    start_events = events_df[events_df['name'].str.contains(r'\d\.start')]
    end_events = events_df[events_df['name'].str.contains(r'\d\.end')]

    # Ensure the events are sorted by timestamp
    start_events = start_events.sort_values(by='timestamp [ns]').reset_index(drop=True)
    end_events = end_events.sort_values(by='timestamp [ns]').reset_index(drop=True)

    # Initialize lists to store results
    event_ids = []
    onsets = []
    durations = []
    nearest_timestamps = []

    for i, __ in start_events.iterrows():
        start_event = start_events.iloc[i]
        end_event = end_events.iloc[i]

        # Calculate onset and duration
        onset = start_event['timestamp [ns]']
        duration = end_event['timestamp [ns]'] - onset

        # Find the nearest timestamp in world_timestamps
        nearest_timestamp = world_timestamps_df.iloc[(world_timestamps_df['timestamp [ns]'] - onset).abs().argmin()]['timestamp [ns]']

        # Append results to lists
        event_ids.append(i + 1)
        onsets.append(onset)
        durations.append(duration)
        nearest_timestamps.append(nearest_timestamp)

    # Create a DataFrame with the results
    result_df = pd.DataFrame({
        'event_id': event_ids,
        'onset_ns': onsets,
        'duration_ns': durations,
        'nearest_timestamp_ns': nearest_timestamps
    })

    return result_df

# Example usage
# aligned_events_df = create_event_onsets_and_durations(events, world_timestamps)
# print(aligned_events_df)


In [ ]:
# testing
aligned_events_df = create_event_onsets_and_durations(events, world_timestamps)
print(aligned_events_df)

### QC Checks

In [5]:
def plot_adjusted_onsets_vs_world_timestamps(aligned_events_df, world_timestamps_df, save_dir):
    # Extract values for plotting
    adjusted_onsets = aligned_events_df['onset_ns']
    world_timestamps = world_timestamps_df['timestamp [ns]']

    # Calculate min and max of adjusted_onsets
    min_onset = adjusted_onsets.min()
    max_onset = adjusted_onsets.max()

    # Define the range within 5 values of min and max
    lower_bound = min_onset 
    upper_bound = max_onset 

    # Filter world_timestamps based on the defined range
    filtered_world_timestamps = world_timestamps[(world_timestamps >= lower_bound) & (world_timestamps <= upper_bound)]

    # print(len(adjusted_onsets))
    # print(len(world_timestamps))
    # print(len(filtered_world_timestamps))

    # Plot adjusted onsets
    plt.figure(figsize=(14, 6))
    plt.scatter(adjusted_onsets, adjusted_onsets, color='red', label='Adjusted Onsets', marker='x')

    # Plot world timestamps
    plt.plot(filtered_world_timestamps, filtered_world_timestamps, label='World Timestamps', alpha=0.7)

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('')
    plt.title('Adjusted Onsets vs. World Timestamps')
    plt.legend()
    
    # # Save the plot
    plot_path = os.path.join(save_dir, 'adjusted_onsets_vs_world_timestamps.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path

# Plot the data
#plot_adjusted_onsets_vs_world_timestamps(aligned_events_df_2, world_timestamps)

# we'll want to update these to save the plots into the correct folder


In [ ]:
plot_adjusted_onsets_vs_world_timestamps(aligned_events_df, world_timestamps, subj_folder)

In [6]:
def plot_nearest_timestamps(aligned_events_df, save_dir):
    # Extract values for plotting
    adjusted_onsets = aligned_events_df['onset_ns']
    nearest_timestamps = aligned_events_df['nearest_timestamp_ns']
    event_ids = aligned_events_df['event_id']

    # Plot nearest timestamps
    plt.figure(figsize=(14, 6))
    plt.scatter(adjusted_onsets, event_ids, color='blue', label='Adjusted Onsets', marker='o', alpha=0.8)
    plt.scatter(nearest_timestamps, event_ids, color='green', label='Nearest Timestamps', marker='x')

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('Event ID')
    plt.title('Adjusted Onsets and Nearest Timestamps per Event')
    plt.legend()
    
    # # Save the plot
    plot_path = os.path.join(save_dir, 'adjusted_onsets_nearest_timestamps.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path

# Plot the nearest timestamps
# plot_nearest_timestamps(aligned_events_df_2)

In [ ]:
plot_nearest_timestamps(aligned_events_df, subj_folder)

In [7]:
def plot_event_durations(aligned_events_df, save_dir):
    # Extract values for plotting
    event_ids = aligned_events_df['event_id']
    adjusted_onsets = aligned_events_df['onset_ns']
    durations = aligned_events_df['duration_ns']

    # Calculate end times based on durations
    adjusted_ends = adjusted_onsets + durations

    # Plot event durations
    plt.figure(figsize=(14, 6))
    for i, event_id in enumerate(event_ids):
        plt.plot([adjusted_onsets[i], adjusted_ends[i]], [event_id, event_id], marker='|', label=f'Event {event_id}' if i == 0 else "")

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('Event ID')
    plt.title('Event Durations and Onsets')
    plt.legend()
    
    # # Save the plot
    plot_path = os.path.join(save_dir, 'event_durations_and_onsets.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path
# Plot the event durations
# plot_event_durations(aligned_events_df_2)


In [ ]:
plot_event_durations(aligned_events_df, subj_folder)

In [ ]:
# New code to handle aligned_events_df_2
# Save frames from aligned events (for QC/testing)
# find frame using pyav
container = av.open(video)
video_stream = container.streams.video[0]

# save_dir = os.path.join(subj_folder, 'ads')
# os.makedirs(save_dir, exist_ok=True)

for index, row in aligned.iterrows():

    event_id = row['event_id']
    nearest_timestamp_ns = row['nearest_timestamp_ns']
    duration = row['duration_ns']

    # Calculate the second timestamp
    second_timestamp_ns = nearest_timestamp_ns + duration

    # Find the frame numbers for both timestamps
    frame_num_1 = np.searchsorted(world_timestamps['timestamp [ns]'], nearest_timestamp_ns)
    frame_num_2 = np.searchsorted(world_timestamps['timestamp [ns]'], second_timestamp_ns)

    frame_num_1 = int(frame_num_1)
    frame_num_2 = int(frame_num_2)

    # Save both frames
    for frame_num in [frame_num_1, frame_num_2]:
        timestamp = frame_num * video_stream.time_base
        container.seek(int(timestamp * av.time_base), stream=video_stream)

        fc = 0
        for frame in container.decode(video=0):
            if fc == frame_num:
                frame = frame.to_ndarray(format="bgr24")
                save_path = os.path.join(save_dir, f'event-{int(event_id)}_{frame_num}.jpg')
                cv2.imwrite(save_path, frame)
                break
                
            fc += 1

## Extract Pupil Labs Videos and Subset Gaze Data

In [8]:
def process_video_segments_phone(video, world_timestamps, events_df, gaze, save_dir):
    # Check the duration of the input video using FFmpeg
    def get_video_duration(video_path):
        ffmpeg_command = [
            'ffprobe',
            '-v', 'error',
            '-show_entries', 'format=duration',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            video_path
        ]
        try:
            duration = subprocess.check_output(ffmpeg_command, stderr=subprocess.STDOUT).decode().strip()
            return float(duration)
        except subprocess.CalledProcessError as e:
            error_message = f"Error getting video duration: {e.output.decode()}"
            print(error_message)
            log_error(error_message)
            return None

    # Get the video duration
    video_duration = get_video_duration(video)
    if video_duration is None:
        raise RuntimeError("Failed to retrieve video duration.")

    # Get the base timestamp to normalize the timeline
    base_timestamp = world_timestamps['timestamp [ns]'].iloc[0]
    # print(f"Base timestamp: {base_timestamp}")

    # Process each ad 
    for index, row in events_df.iterrows():
        event_id = row['event_id']

        ad_save_dir = os.path.join(save_dir, f'{int(event_id)}')
        os.makedirs(ad_save_dir, exist_ok=True)

        nearest_timestamp_ns = row['nearest_timestamp_ns']
        duration = row['duration_ns']

        # Calculate the second timestamp
        second_timestamp_ns = nearest_timestamp_ns + duration

         # Add 0.5 seconds (500 million nanoseconds) before and after the event
        buffer_ns = 500_000_000
        start_timestamp_ns = nearest_timestamp_ns - buffer_ns
        end_timestamp_ns = second_timestamp_ns + buffer_ns

        # Find the frame numbers for both timestamps
        frame_num_1 = np.searchsorted(world_timestamps['timestamp [ns]'], nearest_timestamp_ns)
        frame_num_2 = np.searchsorted(world_timestamps['timestamp [ns]'], second_timestamp_ns)

        frame_num_1 = int(frame_num_1)
        frame_num_2 = int(frame_num_2)

        # Get the start and end timestamps for the ad in nanoseconds
        start_ts = world_timestamps['timestamp [ns]'].iloc[frame_num_1]
        end_ts = world_timestamps['timestamp [ns]'].iloc[frame_num_2]

        # Sort the gaze data by filtering the start and end timestamps
        gaze_data = gaze[(gaze['timestamp [ns]'] >= start_ts) & (gaze['timestamp [ns]'] <= end_ts)]
        gaze_data.to_csv(os.path.join(ad_save_dir, f'gaze.csv'), index=False)

        # Normalize timestamps to video timeline
        normalized_start_ts = start_ts - base_timestamp
        normalized_end_ts = end_ts - base_timestamp

        # Find the corresponding frame numbers
        start_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_start_ts + base_timestamp)
        end_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_end_ts + base_timestamp)

        # Convert normalized timestamps to time in seconds
        start_time = normalized_start_ts / 1e9
        end_time = normalized_end_ts / 1e9
        duration = end_time - start_time

        # Check if the duration matches the duration of the gaze_data
        gaze_duration = (gaze_data['timestamp [ns]'].iloc[-1] - gaze_data['timestamp [ns]'].iloc[0]) / 1e9
        if abs(duration - gaze_duration) > 0.01:
            print(f"Warning: Duration mismatch between video segment and gaze data. Video duration: {duration}, Gaze data duration: {gaze_duration}")

        # Debug: Print calculated times
        # print(f"Event {int(event_id)}: {start_ts} to {end_ts}")
        # print(f"Start Time: {start_time}, End Time: {end_time}, Duration: {duration}")

        # Check if the start time and duration are valid
        if start_time < 0 or duration <= 0 or start_time + duration > video_duration:
            print(f"Invalid time range for {start_ts} to {end_ts}. Skipping.")
            continue

        # Output file path
        output_path = os.path.join(ad_save_dir, f'{int(event_id)}.mp4')

        # FFmpeg command to extract the video segment
        ffmpeg_command = [
            'ffmpeg',
            '-i', video,
            '-ss', f'{start_time:.3f}',  # Start time with millisecond precision
            '-t', f'{duration:.3f}',     # Duration with millisecond precision
            '-c:v', 'libx264',           # Video codec
            '-c:a', 'aac',               # Audio codec
            '-strict', 'experimental',
            '-y',                        # Overwrite output file
            output_path
        ]

        # Execute the FFmpeg command
        try:
            subprocess.run(ffmpeg_command, check=True, stderr=subprocess.PIPE)
            print(f"Segment saved: {output_path}")
        except subprocess.CalledProcessError as e:
            error_message = f"Error extracting segment {start_ts} to {end_ts}: {e.stderr.decode()}"
            print(error_message)
            log_error(error_message)

    print("Video processing completed.")

In [31]:
# testing

process_video_segments_phone(video, world_timestamps_df, aligned, gaze, save_dir)

Segment saved: /Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7/ads/1/1.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7/ads/2/2.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7/ads/3/3.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7/ads/4/4.mp4
Video processing completed.


# Extract Telvision AOI

In [9]:
# attempt 8 # would be cool to have a print out of what % complete it is...idk about that rn (not round here partner, not round here)
def detect_and_save_smartphones(video_path, output_folder, skip_frames=5):
    net = cv2.dnn.readNet("/Users/ebeard/Documents/darknet/yolov3.weights", "/Users/ebeard/Documents/darknet/cfg/yolov3.cfg")
    layer_names = net.getLayerNames()
    classes = []
    with open("/Users/ebeard/Documents/darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    # Handling output layers based on the format of getUnconnectedOutLayers
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if unconnected_out_layers.ndim == 1:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    else:
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]

    cap = cv2.VideoCapture(video_path)
    frame_index = 0
    aoi_data = []

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(os.path.join(output_folder, "frames")):
        os.makedirs(os.path.join(output_folder, "frames"))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if skip_frames == 0 or (frame_index % skip_frames == 0):  # Process this frame
            height, width, channels = frame.shape
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        if classes[class_id].lower() == 'cell phone': 
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            found_phone = False
            largest_box = None
            max_area = 0
            
            if len(indexes) > 0:  # Check if indexes is not empty
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    area = w * h
                    if area > max_area:  # Find the largest bounding box
                        largest_box = (x, y, w, h)
                        max_area = area

                if largest_box is not None:
                    x, y, w, h = largest_box
                    label = str(classes[class_ids[i]])
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    aoi_data.append({'frame': frame_index, 'x': x, 'y': y, 'width': w, 'height': h})
                    found_phone = True
                    
            if found_phone:
                cv2.imwrite(os.path.join(output_folder, "frames", f"frame_{frame_index}.jpg"), frame)

        frame_index += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile: 
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in aoi_data:
            writer.writerow(data)

# Example usage
#aois = detect_and_save_smartphones(video, 'full_frame', skip_frames=100)
#television = detect_and_save_smartphones(video_path = os.path.join('data', 'sub-019_tv-lego-42b0cb7d', '09e25ad2_0.0-784.762.mp4'), output_folder='full_frame_tv', skip_frames=200)


In [ ]:
# testing
detect_and_save_smartphones(ad_video, ad_save_dir, skip_frames=0)

# AOI for each frame

In [10]:
def interpolate_aoi_data_from_video(rectangle_df, video_path, save_dir):
    """
    Interpolates missing AOI data (x, y, width, height) based on the number of frames in the video.
    
    Parameters:
    - rectangle_df (pd.DataFrame): A dataframe containing 'frame', 'x', 'y', 'width', 'height' columns for detected AOIs.
    - video_path (str): Path to the video file to get the total number of frames.
    
    Returns:
    - interpolated_df (pd.DataFrame): A dataframe with interpolated AOI values for each frame in the video.
    """

    # Check if rectangle_df is a file path
    if isinstance(rectangle_df, str):
        rectangle_df = pd.read_csv(rectangle_df)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    
    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()  # Close the video capture

    # Ensure rectangle_df contains only necessary columns and is sorted by frame
    # Calculate the area of each rectangle
    rectangle_df['area'] = rectangle_df['width'] * rectangle_df['height']
    
    # Calculate the IQR for area
    Q1 = rectangle_df['area'].quantile(0.25)
    Q3 = rectangle_df['area'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter out outliers based on the IQR method
    filtered_df = rectangle_df[(rectangle_df['area'] >= lower_bound) & (rectangle_df['area'] <= upper_bound)]

    filtered_df = filtered_df[['frame', 'x', 'y', 'width', 'height']].sort_values(by='frame').reset_index(drop=True)
    
    # Create a DataFrame that has all frame numbers from 0 to total_frames - 1
    all_frames = pd.DataFrame({'frame': np.arange(total_frames)})
    
    # Merge with the existing AOI data (this will insert NaNs for missing AOI data)
    full_frame_df = pd.merge(all_frames, filtered_df, how='left', on='frame')
    
    # Interpolate missing values for x, y, width, height
    full_frame_df[['x', 'y', 'width', 'height']] = full_frame_df[['x', 'y', 'width', 'height']].interpolate(
        method='linear', limit_direction='forward', axis=0)
    
    # Fill any remaining NaNs with the nearest available values
    full_frame_df = full_frame_df.bfill()
    full_frame_df = full_frame_df.ffill()

    # Plot the distribution of the areas and add the lower bound line
    plt.figure(figsize=(8, 6))
    plt.hist(rectangle_df['area'], bins=20, color='blue', edgecolor='black', label='AOI Areas')

    # Add vertical lines for the quartiles and IQR bounds
    plt.axvline(rectangle_df['area'].quantile(0.25), color='red', linestyle='dashed', linewidth=1.5, label="25th Percentile")
    plt.axvline(rectangle_df['area'].quantile(0.50), color='green', linestyle='dashed', linewidth=1.5, label="50th Percentile")
    plt.axvline(rectangle_df['area'].quantile(0.75), color='orange', linestyle='dashed', linewidth=1.5, label="75th Percentile")
    plt.axvline(lower_bound, color='purple', linestyle='dashed', linewidth=1.5, label="Lower Bound (Outliers)")

    plt.title('Distribution of AOI Areas with Lower Bound Line')
    plt.xlabel('Area (width * height)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)

    plot_path = os.path.join(save_dir, 'distribution_of_aois.png')
    plt.savefig(plot_path)
    plt.close()
    
    return full_frame_df

# Example usage:
# Assuming you have a filtered AOI dataframe 'filtered_rectangle_df' and video 'input_video.mp4'
# interpolated_aoi_df = interpolate_aoi_data_from_video(filtered_rectangle_df, 'input_video.mp4')

In [92]:
# testing
rectangle_df = pd.read_csv(os.path.join(save_dir, '6', 'rectangle_data.csv'))
# aoi_df = interpolate_aoi_data_from_video(rectangle_df, ad_video)
# Get the filtered data and lower bound
filtered_aoi_df = interpolate_aoi_data_from_video(rectangle_df, os.path.join(save_dir, '6', '6.mp4'), os.path.join(save_dir, '6'))

# Map Gaze Onto Scene Video, Gaze Within AOI

In [14]:
def calculate_fixation_durations(fixation_data):
    """ Calculate fixation duration based on the number of gaze points for each fixation ID. """
    fixation_data = fixation_data.copy()
    fixation_data.loc[:,'fixation_duration'] = fixation_data.groupby('fixation id')['fixation id'].transform('count')
    return fixation_data

def is_point_in_aoi(gaze_x, gaze_y, aoi_row):
    """ Check if a given gaze point (gaze_x, gaze_y) is inside the AOI rectangle. """
    aoi_x = int(aoi_row['x'])
    aoi_y = int(aoi_row['y'])
    aoi_width = int(aoi_row['width'])
    aoi_height = int(aoi_row['height'])
    
    return aoi_x <= gaze_x <= aoi_x + aoi_width and aoi_y <= gaze_y <= aoi_y + aoi_height

def add_within_aoi_column(gaze_data, aoi_data, video_start_timestamp, frame_duration_ns):
    """ Add a 'within_aoi' column to the gaze data, indicating if the gaze point is within the AOI. """
    within_aoi = []
    
    for _, gaze_row in gaze_data.iterrows():
        gaze_timestamp = gaze_row['timestamp [ns]']
        gaze_x = int(gaze_row['gaze x [px]'])
        gaze_y = int(gaze_row['gaze y [px]'])
        
        # Find the corresponding frame based on the timestamp
        frame_idx = int((gaze_timestamp - video_start_timestamp) // frame_duration_ns)
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        
        if not aoi_row.empty:
            # Check if the gaze point is within the AOI
            within_aoi.append(is_point_in_aoi(gaze_x, gaze_y, aoi_row.iloc[0]))
        else:
            within_aoi.append(False)  # If no AOI for the frame, assume False
    
    gaze_data['within_aoi'] = within_aoi
    return gaze_data

def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path):
    """
    Overlays averaged gaze points (both fixation and non-fixation) onto each frame of a video.
    Scanpaths are drawn between averaged gaze points for continuous movement visualization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, gaze points, and scanpaths overlaid.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Calculate fixation durations based on the number of gaze points per fixation ID
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    fixation_data = calculate_fixation_durations(fixation_data)  # Calculate fixation durations
    
    # Calculate the frame duration based on average_rate
    input_container = av.open(video_path)
    input_stream = input_container.streams.video[0]
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Find the start timestamp of the video
    video_start_timestamp = gaze_data['timestamp [ns]'].min()

    # Add the 'within_aoi' column to the gaze data
    gaze_data = add_within_aoi_column(gaze_data, aoi_data, video_start_timestamp, frame_duration_ns)

    # Define the path for the output video
    new_video_path = os.path.join(output_path, 'output_with_aoi_and_gaze.mp4')
    
    # Open the video file using PyAV
    output_container = av.open(new_video_path, mode='w')
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'

    # Variables to store previous averaged gaze point for scanpath drawing
    prev_gaze_x = None
    prev_gaze_y = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing and to to RGB explicitly (to avoid the warning)
        img = frame.to_image().convert("RGB")

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Find gaze points for the current frame and calculate the average
        gaze_points = gaze_data[(gaze_data['timestamp [ns]'] >= current_frame_timestamp) &
                                (gaze_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        
        if not gaze_points.empty:
            # Average the gaze points for this frame
            avg_gaze_x = int(gaze_points['gaze x [px]'].mean())
            avg_gaze_y = int(gaze_points['gaze y [px]'].mean())
            is_within_aoi = gaze_points['within_aoi'].mean() > 0.5  # If majority of points are inside AOI
            
            # Draw gaze point based on whether it's within AOI
            color = "blue" if is_within_aoi else "orange"
            draw.ellipse([(avg_gaze_x - 3, avg_gaze_y - 3), (avg_gaze_x + 3, avg_gaze_y + 3)], fill=color, outline=color)
            
            # Draw scanpath (line between consecutive averaged gaze points)
            if prev_gaze_x is not None and prev_gaze_y is not None:
                draw.line([(prev_gaze_x, prev_gaze_y), (avg_gaze_x, avg_gaze_y)], fill="blue", width=2)

            # Update the previous gaze point for the next frame
            prev_gaze_x, prev_gaze_y = avg_gaze_x, avg_gaze_y

        # Find fixation points within the current frame's time window and calculate the average
        fixation_rows = fixation_data[(fixation_data['timestamp [ns]'] >= current_frame_timestamp) &
                                      (fixation_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        if not fixation_rows.empty:
            # Average the fixation points for this frame
            avg_fixation_x = int(fixation_rows['gaze x [px]'].mean())
            avg_fixation_y = int(fixation_rows['gaze y [px]'].mean())
            avg_fixation_duration = fixation_rows['fixation_duration'].mean()
            
            # Base size of the circle and scaling it by the averaged fixation duration
            base_size = 3
            scaled_size = base_size + 2 * (avg_fixation_duration ** 0.5)
            radius = int(scaled_size)
            
            # Draw the averaged fixation point with no fill and size proportional to the averaged fixation duration
            draw.ellipse([(avg_fixation_x - radius, avg_fixation_y - radius), (avg_fixation_x + radius, avg_fixation_y + radius)], outline="blue", width=2)

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with averaged gaze points, fixations, and scanpath saved as {new_video_path}")

     # print the updated gaze_data with 'within_aoi' column
    # Save the updated gaze data to a new CSV file
    gaze_data.to_csv(os.path.join(output_path, 'updated_gaze_data.csv'), index=False)

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_averaged_gaze_and_scanpath.mp4', video_start_timestamp)


In [ ]:
# testing
video_gaze_data = os.path.join(save_dir, '7', 'gaze.csv')
overlay_gaze_and_aoi_on_video_ffmpeg(ad_video, video_gaze_data, filtered_aoi_df, os.path.join(save_dir, '7'))

# Loop Through Subjects

In [16]:
for subj in subj_folders[:]:

    try:
        subj_id = subj.split('/')[5].split('_')[0]
        print('------------ Starting processing for ', subj_id,' ------------')
        
        # subj info and files
        subj_folder = subj

        try:
            video = glob.glob(os.path.join(subj, '*.mp4'))[0]
        except IndexError as e:
            error_message = f"Video file not found: {e}"
            print(error_message)
            log_error(error_message, subj=subj)
            continue

        try:
            events_df = pd.read_csv(os.path.join(subj_folder, 'events.csv'))
            world_timestamps_df = pd.read_csv(os.path.join(subj_folder, 'world_timestamps.csv'))
            gaze = pd.read_csv(os.path.join(subj_folder, 'gaze.csv'))
            
        except FileNotFoundError as e:
            error_message = f"CSV file not found: {e}"
            print(error_message)
            log_error(error_message, subj=subj_id)
            continue

        # if we've run this before, skip it
        skip_test = glob.glob(os.path.join(subj, 'ads', '[0-9]', 'output_with_aoi_and_gaze.mp4')) #'[0-9]*'))
        
        if skip_test:
            print(f"Skipping subject as output already exists.")
            continue

        else:
            print('Splitting ads... ')

            save_dir = os.path.join(subj, 'ads')
            os.makedirs(save_dir, exist_ok=True)

            # align pupil labs with openbci data
            try:
                aligned = create_event_onsets_and_durations(events_df, world_timestamps_df)
                aligned.to_csv(os.path.join(save_dir, 'aligned_events.csv'), index=False)
            
            except Exception as e:
                error_message = f"Error aligning events: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            # save open qc plots
            try:
                plot_adjusted_onsets_vs_world_timestamps(aligned, world_timestamps_df, save_dir)
                plot_nearest_timestamps(aligned, save_dir)
                plot_event_durations(aligned, save_dir)
            
            except Exception as e:
                error_message = f"Error generating plots: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            # extract pupil labs videos
            try:
                process_video_segments_phone(video, world_timestamps_df, aligned, gaze, save_dir)
            except Exception as e:
                error_message = f"Error processing video segments: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            # extract telvision AOI
            ads_folders = glob.glob(os.path.join(subj, 'ads', '[0-9]*')) #'[0-9]*'))

            for ad_folder in ads_folders:
                short_video = glob.glob(os.path.join(ad_folder, '*.mp4'))[0]
                event_id = short_video.split('/')[-1].split('.')[0]

                print('Processing ad ', event_id,'...')

                # make output folder
                ad_save_dir = os.path.join(subj, 'ads', event_id)
                os.makedirs(ad_save_dir, exist_ok=True)
                
                # detect tv screens
                try:
                    # if gaze data is longer than 7 seconds, skip frames = 5
                    if aligned[aligned['event_id'] == int(event_id)]['duration_ns'].values[0] > 7e9:
                        skip_frames = 5
                    else:
                        skip_frames = 0
                        
                    detect_and_save_smartphones(short_video, ad_save_dir, skip_frames=0)
                
                except Exception as e:
                    error_message = f"Detecting tv screens: {e}"
                    print(error_message)
                    log_error(error_message, subj=short_video)
                    continue

                # filter rectangles
                try:

                    rectangle_df = os.path.join(ad_folder, 'rectangle_data.csv')
                    aoi_df = interpolate_aoi_data_from_video(rectangle_df, short_video, ad_save_dir)
                    aoi_df.to_csv(os.path.join(ad_folder, 'processed_aoi_for_each_frame.csv'))
                
                except Exception as e:
                    error_message = f"Error filtering rectangles: {e}"
                    print(error_message)
                    log_error(error_message, subj=subj_id)
                    continue

                try:
                    video_gaze_data = os.path.join(ad_folder, 'gaze.csv')
                    overlay_gaze_and_aoi_on_video_ffmpeg(short_video, video_gaze_data, aoi_df, ad_save_dir)
                
                except Exception as e:
                    error_message = f"Error overlaying gaze to video: {e}"
                    print(error_message)
                    log_error(error_message, subj=subj_id)
                    continue


    except Exception as e:
        error_message = f"Unexpected error: {e}"
        print(error_message)
        log_error(error_message, subj=subj_id)

------------ Starting processing for  sub-095  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-096  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-097  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-098  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-099  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-100  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-104  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-105  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-106  ------------
Skipping subject as output already exists.
------------ Starting processing for  sub-107  ------------
Skipping subj

No accelerated colorspace conversion found from yuv420p to rgb24.
timebase 17851/535000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535


Error overlaying gaze to video: [Errno 22] Invalid argument; last error log: [mpeg4] timebase 17851/535000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535
Processing ad  2 ...


No accelerated colorspace conversion found from yuv420p to rgb24.
timebase 17851/535000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535


Error overlaying gaze to video: [Errno 22] Invalid argument; last error log: [mpeg4] timebase 17851/535000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535
Processing ad  3 ...


...

Error overlaying gaze to video: [Errno 22] Invalid argument; last error log: [mpeg4] timebase 964057/28900000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535
Processing ad  6 ...


No accelerated colorspace conversion found from yuv420p to rgb24.
timebase 964057/28900000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535


Error overlaying gaze to video: [Errno 22] Invalid argument; last error log: [mpeg4] timebase 964057/28900000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535
Processing ad  7 ...


No accelerated colorspace conversion found from yuv420p to rgb24.
timebase 964057/28900000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535


Error overlaying gaze to video: [Errno 22] Invalid argument; last error log: [mpeg4] timebase 964057/28900000 not supported by MPEG 4 standard, the maximum admitted value for the timebase denominator is 65535
------------ Starting processing for  sub-192  ------------
Splitting ads... 
Segment saved: /Volumes/T7 Shield/fox/et/sub-192_phone-b-8962a1bf/ads/1/1.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-192_phone-b-8962a1bf/ads/2/2.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-192_phone-b-8962a1bf/ads/3/3.mp4
Error processing video segments: single positional indexer is out-of-bounds
